In [1]:
import pathlib
import pandas as pd
import shutil
import logging

import sys
sys.path.append("../")
from idrstream.DP_idr import DeepProfilerRun

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target,Plate_Map_Name,Gene_Replicate,Site,DNA
0,LT0001_02,A4,50,4,positive control,ENSG00000149503,LT0001_02_4,1,1,LT0001_02/LT0001_02_4_50.tif
1,LT0001_02,A15,50,15,negative control,negative control,LT0001_02_15,1,1,LT0001_02/LT0001_02_15_50.tif
2,LT0001_02,B2,50,26,negative control,negative control,LT0001_02_26,1,1,LT0001_02/LT0001_02_26_50.tif
3,LT0001_02,C1,50,49,positive control,ENSG00000149503,LT0001_02_49,1,1,LT0001_02/LT0001_02_49_50.tif
4,LT0001_02,C4,50,52,positive control,ENSG00000149503,LT0001_02_52,1,1,LT0001_02/LT0001_02_52_50.tif
...,...,...,...,...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11,LT0603_06_349,1,1,LT0603_06/LT0603_06_349_50.tif
6739,LT0603_06,O16,50,352,negative control,negative control,LT0603_06_352,1,1,LT0603_06/LT0603_06_352_50.tif
6740,LT0603_06,P3,50,363,negative control,negative control,LT0603_06_363,1,1,LT0603_06/LT0603_06_363_50.tif
6741,LT0603_06,P21,50,381,positive control,COPB,LT0603_06_381,1,1,LT0603_06/LT0603_06_381_50.tif


In [3]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("../tmp/")
final_data_dir = pathlib.Path("../example_extracted_features/DP_features/")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
except:
    print("No files to remove")

stream = DeepProfilerRun(idr_id, tmp_dir, final_data_dir, log='example_logs/dp_idrstream.log')

No files to remove


In [4]:
# path to users home dir
home_dir_path = pathlib.Path.home()

aspera_path = pathlib.Path(f"{home_dir_path}/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
idr_index_name = "idr0013-neumann-mitocheck"

stream.init_downloader(aspera_path, aspera_key_path, screens_path, idr_index_name)

In [5]:
fiji_path = pathlib.Path(f"{home_dir_path}/Desktop/Fiji.app")
perform_illumination_correction = True
stream.init_preprocessor(fiji_path, perform_illumination_correction)

[INFO] Overriding Leica ROI Reader; identifier: command:de.biovoxxel.utilities.RoiReader; jar: file:/home/roshankern/Desktop/Fiji.app/plugins/Biovoxxel_Plugins-2.5.6.jar


[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [8]:
stream.run_dp_stream(data_to_process, batch_size=3, start_batch=0, batch_nums=[0,1,2])

Completed: 75739K bytes transferred in 2 seconds
 (217459K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00004_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00004/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 87703K bytes transferred in 3 seconds
 (238721K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp/downloads/LT0001_02/00015_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00015/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 74464K bytes transferred in 2 seconds
 (221132K bits/sec), in 1 file.
CellH5Reader initializi

2023-12-28 10:27:49.829538: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
/home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/dataset/metadata.py:38: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.loadSingle(filename, delimiter, dtype)
/home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.


2023-12-28 10:27:50,863 - WARNING - From /home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form ../tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'ENSG00000

2023-12-28 10:27:52.544453: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-12-28 10:27:52.560990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-28 10:27:52.561151: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.882GHz coreCount: 28 deviceMemorySize: 11.74GiB deviceMemoryBandwidth: 335.32GiB/s
2023-12-28 10:27:52.561164: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-12-28 10:27:52.573649: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-12-28 10:27:52.573707: I tensorflow/stream_executor/pl

2023-12-28 10:27:53,695 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

...

2023-12-28 10:29:34,411 - WARNING - From /home/roshankern/anaconda3/envs/idrstream_dp/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form ../tmp/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'negative 

2023-12-28 10:29:36.110050: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-12-28 10:29:36.125278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-28 10:29:36.125448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.882GHz coreCount: 28 deviceMemorySize: 11.74GiB deviceMemoryBandwidth: 335.32GiB/s
2023-12-28 10:29:36.125467: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-12-28 10:29:36.138461: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-12-28 10:29:36.138522: I tensorflow/stream_executor/pl

2023-12-28 10:29:37,272 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream/idrstream/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

2023-12-28 10:29:37.474830: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -5 } dim { size: -6 } dim { size: -7 } dim { size: 1 } } } inputs { dtype: DT_FLOAT shape { dim { size: -2 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -2 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } tensor_content: "\200\000\000\000\200\000\000\000" } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 3060" frequency: 1882 num_cores: 28 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 2359296 shared_memory_size_

LT0001_02/74_50-1 (299 cells) : 3.52 secs
LT0001_02/304_50-1 (173 cells) : 1.11 secs
LT0001_02/315_50-1 (151 cells) : 1.02 secs
Profiling: done
